In [ ]:
! pip install -r '/home/jupyter/sample_project/requirements.txt'

In [8]:
import pandas as pd
import gcsfs
import recordlinkage as rl
import matching as m
import helpers as h
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from gcsfs.core import GCSFileSystem
gcs = GCSFileSystem('courseradataeng', token='/home/jupyter/.config/gcloud/application_default_credentials.json') 

In [10]:
with gcs.open('soleadify_sample_data/website_dataset_clean.csv') as f:
    website_dataset_clean = pd.read_csv(f)


In [14]:
with gcs.open('soleadify_sample_data/google_dataset_clean.csv') as f:
    google_dataset_clean = (pd.read_csv(f , quotechar='"', escapechar='\\')
                                .drop(['Unnamed: 0','region_name'],axis=1)
                                .rename(columns={'country_final':'country_clean'
                                        ,'region_consistent':'region_clean'
                                       ,'city_consistent':'city_clean'
                                       ,'domain':'root_domain'
                                       ,'name':'site_name_clean'}
                                       )
                             )

In [15]:
levels_of_logic = {
    'level_1':{'country_clean':'value'
              ,'region_clean':'value'
              ,'city_clean':'value'
              ,'root_domain':'notna'
              ,'site_name_clean':'notna'
             }
    ,'level_2':{'country_clean':'value'
              ,'region_clean':'value'
              ,'city_clean':'isna'
              ,'root_domain':'notna'
              ,'site_name_clean':'notna'
             }
    ,'level_3':{'country_clean':'value'
              ,'region_clean':'isna'
              ,'city_clean':'isna'
              ,'root_domain':'notna'
              ,'site_name_clean':'notna'
             }
    ,'level_4':{'country_clean':'isna'
              ,'region_clean':'isna'
              ,'city_clean':'isna'
              ,'root_domain':'notna'
              ,'site_name_clean':'notna'
             }
    ,'level_5':{'country_clean':'isna'
              ,'region_clean':'isna'
              ,'city_clean':'isna'
              ,'root_domain':'notna'
              ,'site_name_clean':'isna'
             }
}
origin_mapping = {'dataset_1':'google','dataset_2':'website'}
dataset_1 = google_dataset_clean
dataset_2 = website_dataset_clean

In [16]:
index = 0
for logic_dict in levels_of_logic.values():
    blocking_columns = []
    similar_columns = []
    for column in logic_dict.keys():
        if logic_dict[column] == 'value':
            blocking_columns.append(column)
        elif logic_dict[column] == 'notna':
            similar_columns.append(column)
    
    combinations = h.get_combinations(dataset_1, dataset_2, blocking_columns)
    
    if combinations:

        for combination in combinations[:2]:
            subset_of_matches = m.match(dataset_1, dataset_2, origin_mapping, similar_columns, logic_dict, combination)
            if not subset_of_matches.empty:
                file_name = 'subset_of_matches_'+str(index)+'.csv'
                with gcs.open('soleadify_sample_data/google_website_matches/'+file_name,'w') as f:
                    subset_of_matches.to_csv(f, index = False, quotechar='"', escapechar='\\')
                
                index += 1
                print(index)
    else:
        
        combination = combinations
        subset_of_matches = m.match(dataset_1, dataset_2, origin_mapping, similar_columns, logic_dict, combination)
        if not subset_of_matches.empty:

            file_name = 'subset_of_matches_'+str(index)+'.csv'
            with gcs.open('soleadify_sample_data/google_website_matches/'+file_name,'w') as f:
                subset_of_matches.to_csv(f, index = False, quotechar='"', escapechar='\\')
        
            index += 1
            print(index)
        

1
2


/opt/conda/lib/python3.7/site-packages/recordlinkage/base.py:139: RuntimeWarning: invalid value encountered in long_scalars
  rr = 1 - n / n_max
/opt/conda/lib/python3.7/site-packages/recordlinkage/base.py:155: RuntimeWarning: invalid value encountered in long_scalars
  rr_avg = 1 - n_total / n_max_total


3
4
5
6
